Create category weights for each. Points distribution from 1 to 10. Use industry group data point median as 5.

"The question of materiality, or in other words, the relative weight, is determined by the relative median value for
a company in that industry group. The relative median values for each industry group to which the data point is
material are compared, and decile ranks are assigned. The decile rank determines the relative weight
assigned to that data point in determining the industry weight – from 1 to 10"

<center><span style="font-size:30px; font-weight: bold;">Nordic Compass Database</span></center>
<center><span style="font-size:24px;">Analysis of Environmental Performance and CSRD Compliance</span></center>

<center><span style="font-size:22px;"><b>Section 3:</b> Impact analysis </span></center>

## Introduction to this section

## Imports

In [2]:
import pandas as pd
import numpy as np
import sys
import os

pd.set_option("display.max_columns", None)
sys.path.append(os.path.abspath(".."))
# import random

from functions import test_company, show_missing_values
from IPython.display import display

pd.options.display.float_format = "{:,.2f}".format

## Impact (materiality)

In [3]:
impact_df = pd.read_csv("../datasets/impact_df_original.csv")

In [4]:
impact_df.describe()

,year,csrd_2025,csrd_2027,base_year,external_audit_of_ESG_report,revenue_MEUR,energy_consump_GJ,water_withdraw_thm3,ghg_emis_kt,transport_emis_kt,ghg_emis_per_MEUR_revenue,water_withdraw_per_MEUR_revenue
count,"1,837.00","1,837.00","1,837.00","1,837.00","1,837.00","1,802.00","1,109.00",648.00,"1,232.00",946.00,"1,228.00",646.00
mean,"2,020.43",0.61,0.33,"2,019.13",0.40,"2,476.33","11,051,733.20","259,321.08","6,633,063.26","1,290,251.09","2,396.50",614.96
std,1.11,0.49,0.47,0.51,0.49,"7,014.54","48,349,169.42","2,979,205.87","152,416,032.97","39,340,312.81","53,105.08","12,379.17"
min,"2,019.00",0.00,0.00,"2,019.00",0.00,0.03,0.05,0.00,0.00,0.00,0.00,0.00
25%,"2,019.00",0.00,0.00,"2,019.00",0.00,217.10,"60,449.40",100.89,1.89,2.62,0.00,0.07
50%,"2,020.00",1.00,0.00,"2,019.00",0.00,601.58,"423,277.92",562.18,16.37,25.03,0.01,0.41
75%,"2,021.00",1.00,1.00,"2,019.00",1.00,"1,912.40","2,420,142.00","5,754.50",137.67,389.67,0.07,4.27
max,"2,022.00",1.00,1.00,"2,022.00",1.00,"143,208.96","489,600,000.00","63,372,912.00","4,834,768,000.00","1,210,000,000.00","1,653,973.29","312,843.68"


In [5]:
impact_df.head()

,company,ticker,year,csrd_2025,csrd_2027,segment,industry,hq_country,base_year,external_audit_of_ESG_report,revenue_MEUR,energy_consump_GJ,water_withdraw_thm3,ghg_emis_kt,transport_emis_kt,ghg_emis_per_MEUR_revenue,water_withdraw_per_MEUR_revenue
0,Archer Ltd.,ARCHO,2020,1,0,Mid,Energy,Norway,2020,1,735.71,"459,927.00",NaN,NaN,NaN,NaN,NaN
1,AutoStore Holdings Ltd.,AUTO,2021,0,1,Large,Industrial Goods and Services,Bermuda,2021,0,292.50,NaN,NaN,0.74,371.92,0.00,NaN
2,Avance Gas Holding ltd,AGAS,2019,0,1,Mid,Energy,Norway,2019,0,223.59,NaN,NaN,NaN,NaN,NaN,NaN
3,Avance Gas Holding ltd,AGAS,2020,0,0,Mid,Energy,Norway,2019,1,183.68,"5,934,145.00",NaN,NaN,NaN,NaN,NaN
4,Borr Drilling Ltd,BDRILL,2019,1,0,Mid,Energy,Bermuda,2019,0,291.85,"1,980,428.40",NaN,150.78,43.67,0.52,NaN


## Further cleaning

In [6]:
impact_df.groupby("segment")["revenue_MEUR"].describe()

,count,mean,std,min,25%,50%,75%,max
segment,,,,,,,,
Large,"1,015.00","3,987.52","9,022.35",1.95,409.41,"1,313.80","3,726.17","143,208.96"
Mid,777.00,527.22,992.48,0.03,102.82,304.23,649.11,"20,718.00"
Small,10.00,537.90,428.82,12.28,108.39,602.97,823.31,"1,194.64"


# First, visualise...

In [7]:
from typing import Tuple, Union, Optional, List, Dict


def calculate_bins_for_eda(
    data: pd.Series,
) -> Tuple[Union[float, int], Union[float, int], Union[float, int], int]:
    """
    Calculates bin parameters for numeric and categorical data.

    Parameters:
        data (pd.Series): A Pandas Series (column) to calculate bins for.

    Returns:
        Tuple[Union[float, int], Union[float, int], Union[float, int], int]:
            - bin_start (float or int): The start of the bins (always 0 for numeric data).
            - bin_end (float or int): The end of the bins.
            - bin_size (float or int): The size of each bin (always 1 for categorical data).
            - n_bins (int): The number of bins.
    """
    if isinstance(data.dtype, pd.CategoricalDtype) or pd.api.types.is_object_dtype(
        data
    ):
        unique_values: int = len(data.unique())
        bin_start: int = 0
        bin_end: int = unique_values
        bin_size: int = 1
        n_bins: int = unique_values
    else:
        data = data.dropna()
        if data.empty:
            return 0, 1, 1, 1

        data_min: float = max(0, data.min())
        data_max: float = data.max()
        data_range: float = data_max - data_min

        n_bins: int = min(50, len(data.unique()))  # maximum of 20 bins
        bin_size: float = max(1, round(data_range / n_bins))  # avoid zero-sized bins

        # Adjust bin size to a more rounded number (e.g., 1, 2, 5, 10, etc.)
        if bin_size >= 10:
            scale = 10 ** (len(str(int(bin_size))) - 1)
            bin_size = round(bin_size / scale) * scale

        bin_start: float = 0
        bin_end: float = np.ceil(data_max / bin_size) * bin_size

    return bin_start, bin_end, bin_size, n_bins

In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define discrete columns outside the function for flexibility
columns_to_include = [
    # "year",
    # "segment",
    # "industry",
    # "hq_country",
    # "external_audit_of_ESG_report",
    # "base_year",
    "revenue_MEUR",
    "energy_consump_GJ",
    "water_withdraw_thm3",
    "ghg_emis_kt",
    "transport_emis_kt",
    "ghg_emis_per_MEUR_revenue",
    "water_withdraw_per_MEUR_revenue",
]


def chart_visualisations(
    df: pd.DataFrame,
    columns_to_include: list,
    legend_column: str = None,
    n_cols: int = 3,
) -> go.Figure:
    """
    Creates a 3-column subplot visualization for the provided DataFrame columns,
    displaying a histogram (with quantile-defined bins) for each column.

    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame containing the data to visualize.

    columns_to_include : list
        A list of columns to visualize.

    legend_column : str, optional
        The name of the column that differentiates the data groups. If None, no grouping is applied.

    n_cols : int, optional
        The number of columns in the subplot layout.

    Returns:
    -------
    go.Figure
        The Plotly figure object containing the histograms.
    """
    relevant_columns = [col for col in columns_to_include if col in df.columns]
    n_rows = -(-len(relevant_columns) // n_cols)
    subplot_titles = [f"{col}" for col in relevant_columns]

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=subplot_titles,
    )

    for idx, col in enumerate(relevant_columns):
        row_num = (idx // n_cols) + 1
        col_num = (idx % n_cols) + 1

        data = pd.to_numeric(df[col], errors="coerce").dropna()
        if data.empty:
            continue

        bin_start, bin_end, bin_size, n_bins = calculate_bins_for_eda(data)
        bins = np.arange(bin_start, bin_end + bin_size, bin_size)

        # Fix: np.histogram should count occurrences, not sum values
        counts, _ = np.histogram(data, bins=bins)

        bin_centers = (bins[:-1] + bins[1:]) / 2
        widths = bins[1:] - bins[:-1]

        fig.add_trace(
            go.Bar(
                x=bin_centers,
                y=counts,  # Ensure we're plotting counts, not sums
                width=widths,
                name=f"{col} Distribution",
                showlegend=False,
            ),
            row=row_num,
            col=col_num,
        )

        fig.update_yaxes(title_text="Count", row=row_num, col=col_num, showgrid=False)

    fig.update_layout(
        bargap=0,
        showlegend=False,
        legend_title_text=legend_column,
        height=400 * n_rows,
        width=1600,
        template="plotly_white",
    )

    return fig

In [9]:
chart_visualisations(impact_df, columns_to_include=columns_to_include)

Star Bulk Carriers - Rev. --> 821.365M, not 821,000
Telenor - 9,799M
Cloetta - 649.106

In [10]:
impact_df[impact_df["revenue_MEUR"] > 40000].sort_values(
    by="revenue_MEUR", ascending=False
)

,company,ticker,year,csrd_2025,csrd_2027,segment,industry,hq_country,base_year,external_audit_of_ESG_report,revenue_MEUR,energy_consump_GJ,water_withdraw_thm3,ghg_emis_kt,transport_emis_kt,ghg_emis_per_MEUR_revenue,water_withdraw_per_MEUR_revenue
811,Equinor ASA (formerly Statoil ASA),EQNR,2022,1,0,Large,Energy,Norway,2019,1,"143,208.96",NaN,"6,000.00","11,400.00","243,000.00",0.08,0.04
396,Fortum Oyj,FORTUM,2021,1,0,Large,Utilities,Finland,2019,1,"112,400.00","399,600,000.00","12,359,000.00","69,750.70","120,228.00",0.62,109.96
810,Equinor ASA (formerly Statoil ASA),EQNR,2021,1,0,Large,Energy,Norway,2019,1,"79,235.71","212,400,000.00","8,000.00","12,100.00","249,000.00",0.15,0.10
103,A.P. Møller -Maersk A/S,MAERSK,2022,1,0,Large,Industrial Goods and Services,Denmark,2019,0,"77,425.45","447,345,000.00",916.00,"34,506.00","43,451.00",0.45,0.01
808,Equinor ASA (formerly Statoil ASA),EQNR,2019,1,0,Large,Energy,Norway,2019,1,"56,170.54","252,000,000.00","12,000.00","14,900.00","247,000.00",0.27,0.21
102,A.P. Møller -Maersk A/S,MAERSK,2021,1,0,Large,Industrial Goods and Services,Denmark,2019,1,"55,166.96","473,188,000.00","1,834.00","37,173.00","28,952.00",0.67,0.03
395,Fortum Oyj,FORTUM,2020,1,0,Large,Utilities,Finland,2019,1,"49,015.00","489,600,000.00","4,967,000.00","49,632.00","27,836.40",1.01,101.34
593,AstraZeneca PLC,AZN,2022,1,0,Large,Health Care,United Kingdom,2019,0,"42,118.71","588,963.60","3,750.00",440.24,"5,884.00",0.01,0.09
1062,Volvo AB,VOLV,2019,1,0,Large,Industrial Goods and Services,Sweden,2019,1,"41,133.12","7,624,800.00","5,706.00",324.00,NaN,0.01,0.14
809,Equinor ASA (formerly Statoil ASA),EQNR,2020,1,0,Large,Energy,Norway,2019,1,"40,850.89","234,000,000.00","8,000.00","13,500.00","250,000.00",0.33,0.20


Create a column: 'GHG per EUR revenue_ranking_all_companies' - This is binned from 1 to 10 (using quartiles and calculated using only values 
from the same year)

Create a column: 'GHG per EUR revenue_ranking_sector' - This is also binned from 1 to 10 (and calculated using only values from the same year)

Calculate the average GHG per EUR revenue as well as IQR--apply the outlier transformation and put all outliers in the '0' bin

Create a column: 'GHG per EUR revenue_ranking_all_PY' -- This is to compare to the values from the previous year

Create a column: 'GHG per EUR revenue_ranking_sector_PY' -- This is to compare to the values from the previous year

Create a column: '% change in GHG per EUR revenue vs PY'

Create a column: '% change in GHG emissions vs PY'

Create a column: 'transport emissions as a % of total emissions'

Create a column: '% change in transport emissions vs PY'

Create a column: 'Transport emissions as % of total emissions' (compare to sector)


Use the bins only for GHG emissions/EUR--compare values in each bin for all columns...

See how bin values vary from year to year

Calculate the number of companies that have migrated from bin to bin





#### Bonus columns

'GHG intensity reduction % vs sector-specific targets'--normalise so make it a % above or below target

'GHG intensity reduction % vs others in the sector_CY'--also normalise (and consider whether positive is good or bad)


